Pulling list of applicable races, then using the PCS python API (unofficial) to pull results for those races for the last X years. Applicable races will be [UCI Pro and above](https://www.procyclingstats.com/races.php?season=2023&category=1&racelevel=2&pracelevel=smallerorequal&racenation=&class=&filter=Filter&p=uci&s=calendar-plus-filters), plus [national champs](https://www.procyclingstats.com/races.php?season=2023&category=1&racelevel=3&pracelevel=smallerorequal&racenation=&class=NC&filter=Filter&p=uci&s=calendar-plus-filters)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from procyclingstats import Race, Stage
import ast
from tqdm import tqdm


def get_races(year):
    '''
    Function to get a list of race URLs for a given year
    '''
    # URLs of the page to scrape - Pro Tour up and national championships
    prefix = 'https://www.procyclingstats.com/races.php?season='
    suffixes = [
        '&category=1&racelevel=3&pracelevel=smallerorequal&racenation=&class=NC&filter=Filter&p=uci&s=calendar-plus-filters',
        '&category=1&racelevel=2&pracelevel=smallerorequal&racenation=&class=&filter=Filter&p=uci&s=calendar-plus-filters'
    ]
    urls = [prefix + str(year) + suffix for suffix in suffixes]

    # Iterate over the URLs
    race_urls = []
    for url in urls:
        # Send a GET request to the page
        response = requests.get(url)
        response.raise_for_status()

        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all race entries
        race_entries = soup.select('table tr a[href]')

        # Extract race names and URLs
        for entry in race_entries:
            race_url = 'https://www.procyclingstats.com' + entry['href']
            race_urls.append(race_url)

    race_urls = list(set(race_urls)) # Drop duplicates, in case
    race_urls = [race_url[31:] for race_url in race_urls] # Drop the beginning boilerplate from the URL - API doesn't need it

    return race_urls

def get_race_info(race_url, verbose=True):
    '''
    Function to get a list of stage URLs plus extra variables from the race page
    https://procyclingstats.readthedocs.io/en/latest/api.html#procyclingstats.race_scraper.Race
    Returns a dataframe with a row for each stage
    '''
    # TODO this should use a race_variables var to set variables, as in the below functions
    race = Race(race_url)
    stage_urls = {
        'stage_url': [d['stage_url'] for d in race.stages()] or [f"{race_url}/result"] # elegance
    }
    extra_vars = {
        'race_name': [race.name()],
        'race_category': [race.category()],  
        'uci_tour': [race.uci_tour()]
    }
    if(verbose):
        print(f"{extra_vars['race_name'][0]}")
    output = pd.merge(pd.DataFrame(extra_vars), pd.DataFrame(stage_urls), how='cross')
    return output

def fill_stage_info(race_df):
    '''
    Function to fill in the extra variables for each stage.
    Iterates through URLs. Unpacking results happens later.
    '''
    # Drop troubling races
    race_df = race_df[~race_df['stage_url'].str.contains('qinghai')]

    # List of variables we want from each stage
    stage_variables = ['is_one_day_race', 'distance', 'stage_type', 'winning_attack_length', 
                       'date', 'won_how', 'avg_speed_winner', 'avg_temperature', 
                       'vertical_meters', 'profile_icon', 'profile_score', 
                       'race_startlist_quality_score', 'results','gc','points','kom','youth','teams']
    
    # Preallocate an empty DataFrame with the same number of rows as race_df
    stage_info_df = pd.DataFrame(index=race_df.index, columns=stage_variables)
    
    for i, stage_url in tqdm(enumerate(race_df['stage_url'])):
        # print(f"Filling stage info for {stage_url}")

        # Initialize selected_stage as empty
        selected_stage = pd.DataFrame({key: [None] for key in stage_variables})

        try:
            stage = Stage(stage_url)
        except:
            print(f"Error retrieving stage {i} at {stage_url}")
            # Assign the collected information to the preallocated DataFrame
            stage_info_df.iloc[i] = selected_stage.iloc[0]
            continue
        
        try:
            # Parse the stage information
            parsed_stage = stage.parse()
            selected_stage.update({key: [parsed_stage[key]] for key in parsed_stage if key in stage_variables})
        except Exception as e:
            print(f"Error parsing stage {i} at {stage_url}: {e}")
        
        try:
            # Get results
            selected_stage['results'] = [stage.results()]
        except Exception as e:
            print(f"Error retrieving results for stage {i} at {stage_url}: {e}")
        
        # Assign the collected information to the preallocated DataFrame
        stage_info_df.iloc[i] = selected_stage.iloc[0]
    
    # Concatenate the original race_df with the stage_info_df
    combined_df = pd.concat([race_df, stage_info_df], axis=1)
    
    return combined_df

def expand_results(filled_stage_df):
    '''
    Function to expand the main results column + secondary results columns
    '''
    # list of variables we want from main results
    result_variables = ['rider_name', 'rider_url', 'team_name', 'team_url', 'rank','status','time','uci_points','pcs_points', 'age']
    secondary_result_types = ['gc', 'points', 'kom', 'youth']  # 'teams' is not included here - need to add later as can't join onto rider_url
    secondary_result_variables = ['rank','uci_points']
    
    df = pd.DataFrame(columns= list(filled_stage_df).extend(result_variables))

    # Expand main results (stage-rider level)
    print("Expanding main results")
    for i, row in tqdm(filled_stage_df.iterrows(),total=filled_stage_df.shape[0]):
        results = row['results']
        stage_info = pd.DataFrame(row).T.reset_index(drop=True)

        if isinstance(results, str):
            results = ast.literal_eval(results)
        if not isinstance(results, list):
            continue

        # Expand main results
        for result in results:
            selected_result = pd.DataFrame({key: [value] for key, value in result.items() if key in result_variables})
            full_row = pd.concat([stage_info, selected_result], axis=1).drop(columns=['results'])
            df = pd.concat([df, full_row], ignore_index=True)

    # Expand secondary results (gc, points, kom, youth, teams)
    print("Expanding secondary results")
    for result_type in secondary_result_types:
        expanded_secondary_results = pd.DataFrame()  # Temp dataframe to store expanded rows for each type

        for i, row in tqdm(filled_stage_df.iterrows()):
            results = row[result_type]
            if isinstance(results, str):
                results = ast.literal_eval(results)
            if not isinstance(results, list):
                continue

            for result in results:
                selected_result = pd.DataFrame(
                    {f"{result_type}_{key}": [value] for key, value in result.items() if key in secondary_result_variables}
                )
                selected_result['stage_url'] = row['stage_url']
                selected_result['rider_url'] = result.get('rider_url')
                expanded_secondary_results = pd.concat([expanded_secondary_results, selected_result], ignore_index=True)

        # Merge the accumulated secondary results for the current type into the main dataframe
        df = pd.merge(df, expanded_secondary_results, on=['stage_url', 'rider_url'], how='left').drop(result_type, axis=1)

    df = df.drop('teams', axis=1)  # Drop teams for now

    return df

def pull_race_data(years):
    '''
    Function to pull all data for a given year or years
    '''
    combined_df = pd.DataFrame()
    for year in years:
        df = pd.DataFrame()
        print(f"Pulling races for {year}")
        race_urls = get_races(year)
        for race_url in tqdm(race_urls):
            df = pd.concat([df, get_race_info(race_url,verbose=False)], ignore_index=True)
        df.to_csv(f"race_df_{year}.csv", index=False)
        print(f"Filling stages for {year}")
        df = fill_stage_info(df)
        df.to_csv(f"stage_df_{year}.csv", index=False)
        print(f"Expanding results for {year}")
        df = expand_results(df)
        df.to_csv(f"results_df_{year}.csv", index=False)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    combined_df.to_csv(f"results_df_{years[0]}_{years[-1]}.csv", index=False)
    return combined_df

In [4]:
# Run whole setup for 2018-2020 with saves
results_df_2018_2020 = pull_race_data([2018,2019,2020])

Pulling races for 2018


100%|██████████| 191/191 [02:19<00:00,  1.37it/s]


Filling stages for 2018


2it [00:01,  1.23it/s]

Error parsing stage 1 at race/nc-cyprus-itt/2018/result: could not convert string to float: '-'


...

346it [04:58,  1.16it/s]


Expanding results for 2020
Expanding main results


100%|██████████| 346/346 [00:54<00:00,  6.39it/s]


Expanding secondary results


346it [00:05, 60.92it/s] 
346it [00:01, 226.14it/s]
346it [00:00, 472.88it/s]
346it [00:01, 334.76it/s]
